# Making your dataset
This notebook contains some logic that can be used to format a text file of conversations to the format required by PersonaChat.
The text file should have conversations made of lines of dialogue alternating between two speakers.  
Each utterance of dialogue should be on a separate line.
The conversations should be separated by a single newline.

In [26]:
import json, pickle
from random import choice

In [27]:
with open('conversations.txt') as txt:
    data = txt.read()

convos = data.split('\n\n')
convo_list = []

for convo in convos:
    fixed = convo.lower()
    fixed = fixed.replace('.', ' .').replace(',', '').replace('?', ' ?').replace('!', ' !')
    convo_list.append(fixed.split('\n'))

convo_list[1]

['hi i want to close my account .',
 'you wish to close an existing account . is that correct ?',
 "that's right .",
 'i can log a request to speed up the process . please give me your first and last name .',
 'my name is <f_name> <l_name>',
 'your first and last name is <f_name> <l_name> . is that correct ?',
 'correct .',
 'a request has been logged . is there anything else i can help with ?',
 "that's all .",
 'thank you for using botbank .']

In [28]:
with open('financial_responses.txt') as e_file:
    extras = e_file.readlines()
extras[:5]

['but it depends on the circumstances and what it is you want to deduct\n',
 'travel taxpayers who travel away from home on business may deduct related expenses including the cost of reaching their destination the cost of lodging and meals and other ordinary and necessary expenses\n',
 'taxpayers are considered traveling away from home if their duties require them to be away from home substantially longer than an ordinary days work and they need to sleep or rest to meet the demands of their work\n',
 'the actual cost of meals and incidental expenses may be deducted or the taxpayer may use a standard meal allowance and reduced record keeping requirements\n',
 'regardless of the method used meal deductions are generally limited to 50 percent as stated earlier\n']

In [29]:
e_trim = []
for thing in extras:
    # Choose a sequence of arbitrary length
    # In this instance, only sequences between 40 and 100 characters were kept
    if len(thing) < 100 and len(thing) > 40:
        e_trim.append(thing.replace('\n', ''))
e_trim[:5]

['but it depends on the circumstances and what it is you want to deduct',
 'expenses must be reasonable and appropriate deductions for extravagant expenses are not allowable',
 'more information is available in publication 463 travel entertainment gift and car expenses',
 'more discussion of the rules and limitations can be found in publication 463',
 'edit for meal expenses amount of standard meal allowance']

In [30]:
personality = [
    "i am here help you with your questions and requests .",
    "i am a customer support helper for BotBank .",
    "botbank is a globally trusted financial institution .",
    "botbank offers a wide variety of financial services .",
    "i am a customer support engine .",
    "i cannnot do some things that my human counter parts can but i can still help ."
]

In [31]:
len(convo_list)

30

In [34]:
train_data = {}
train = []
for i in range(24):
    helper = {}
    convo = convo_list[i]
    helper['personality'] = personality
    utts = []
    for i in range(0,len(convo)-1,2):
        utterance = {}
        utterance['candidates'] = [choice(e_trim) for i in range(5)]
        utterance['candidates'].append(convo[i+1])
        utterance['history'] = convo[:i+1]
        utts.append(utterance)
    helper['utterances'] = utts
    train.append(helper)

validate = []
for i in range(24, 30):
    helper = {}
    convo = convo_list[i]
    helper['personality'] = personality
    utts = []
    for i in range(0,len(convo)-1,2):
        utterance = {}
        utterance['candidates'] = [choice(e_trim) for i in range(5)]
        utterance['candidates'].append(convo[i+1])
        utterance['history'] = convo[:i+1]
        utts.append(utterance)
    helper['utterances'] = utts
    validate.append(helper)

train_data['train'] = train
train_data['validate'] = validate


In [35]:
with open('training_data.json', 'w') as file:
    json.dump(train_data, file)